# Car Price prediction

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот ноутбук является шаблоном (Baseline) к текущему соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **Baseline** создается больше как шаблон, где можно посмотреть, как происходит обращение с входящими данными и что нужно получить на выходе. При этом ML начинка может быть достаточно простой. Это помогает быстрее приступить к самому ML, а не тратить ценное время на инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если наше решение хуже baseline -  мы явно делаем что-то не так и стоит попробовать другой путь) 

## В baseline мы сделаем следующее:
* Построим "наивную"/baseline модель, предсказывающую цену по модели и году выпуска (с ней будем сравнивать другие модели)
* Обработаем и отнормируем признаки
* Сделаем первую модель на основе градиентного бустинга с помощью CatBoost
* Сделаем вторую модель на основе нейронных сетей и сравним результаты
* Сделаем multi-input нейронную сеть для анализа табличных данных и текста одновременно
* Добавим в multi-input сеть обработку изображений
* Осуществим ансамблирование градиентного бустинга и нейронной сети (усреднение их предсказаний)

In [2]:
!pip install -q tensorflow==2.3

In [3]:
#аугментации изображений
!pip install albumentations -q

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
import albumentations
from sklearn.feature_selection import f_regression, mutual_info_regression
# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
np.set_printoptions(suppress=True)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [7]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [8]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [9]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [10]:
train.info()

In [11]:
train.nunique()

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [12]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [13]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

Посмотрим, как выглядят распределения числовых признаков:

In [14]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Итого:
* CatBoost сможет работать с признаками и в таком виде, но для нейросети нужны нормированные данные.

# PreProc Tabular Data

In [15]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']

In [16]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

In [17]:
data.info()

In [18]:
# Посмотрим на распределения целевой переменной и логарифмированной ее версии
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.title('price')
sns.distplot(data[data['sample'] == 1].price)
plt.subplot(1, 2, 2)
plt.title('log_price')
sns.distplot(np.log(data[data['sample'] == 1].price))

In [ ]:
# Так как целевая переменная имеет тяжелый правый хвост, а логарифмирование приводит ее к более нормальному распредению то для CatBoost буду использовать логарифм.

In [ ]:
# Займемся предобработкой данных

In [25]:
# bodyType
data.bodyType.value_counts()

In [26]:
# brand
data.brand.value_counts()

In [27]:
#color
data.color.value_counts()

In [22]:
data.engineDisplacement.value_counts()

In [23]:
# Переведем признак engineDisplacement из категориального в числовой дополнительно взяв логарифм для более нормального распределения
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: '2.0 LTR' if x == 'undefined LTR' else x)
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.title('engineDisplacement')
sns.distplot(data['engineDisplacement'].apply(lambda x: float((x.split(' ')[0]))))
plt.subplot(1, 2, 2)
plt.title('log_engineDisplacement')
sns.distplot(np.log(data['engineDisplacement'].apply(lambda x: float((x.split(' ')[0])))))

In [ ]:
# В моделях будем использовать признак engineDisplacement как числовой

In [28]:
# enginePower
data.enginePower.value_counts()

In [29]:
# Переведем признак enginePower из категориального в числовой дополнительно взяв логарифм для более нормального распределения
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.title('enginePower')
sns.distplot(data['enginePower'].str.split().apply(lambda x: int(x[0])))
plt.subplot(1, 2, 2)
plt.title('log_enginePower')
sns.distplot(np.log(data['enginePower'].str.split().apply(lambda x: int(x[0]))))

In [ ]:
# В моделях будем использовать признак enginePower как числовой

In [30]:
# fuelType
data.fuelType.value_counts()

In [31]:
# model_info
data.model_info.value_counts()

In [32]:
# name
data.name.value_counts()

In [33]:
#Понизим размерность name убрав из признака дублирующуюся информацию из колонок 'enginePower', 'engineDisplacement', 'vehicleTransmission'
data.name.str.split(' ').apply(lambda x: x[0]).value_counts()

In [ ]:
# В моделях будем использовать name c меньшей размерностью


In [34]:
#добавим новый признак  4wd из данных колонки name
data['name'].apply(lambda x: 1 if '4WD' in x else 0).value_counts()


In [35]:
#добавим новый признак  4wd из данных колонки name
data['name'].apply(lambda x: 1 if 'xDrive' in x else 0).value_counts()

In [36]:
# numberOfDoors
data.numberOfDoors.value_counts()

In [37]:
# vehicleTransmission
data.vehicleTransmission.value_counts()

In [38]:
# Владельцы
data.Владельцы.value_counts()

In [39]:
# ПТС
data.ПТС.value_counts()

In [40]:
# Привод
data.Привод.value_counts()

In [41]:
# Руль
data.Руль.value_counts()

In [42]:
# Посмотрим на распредение признака mileage и его логарифмированной версии
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.title('mileage')
sns.distplot(data['mileage'])
plt.subplot(1, 2, 2)
plt.title('log_mileage')
sns.distplot(np.log(data['mileage']))

In [ ]:
# Логарифмирование не улучшило распредение потому оставим признак без изменений

In [43]:
# Посмотрим на распредение признака modelDate и его логарифмированной версии
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('mileage')
sns.distplot(2021 - data['modelDate'])
plt.subplot(1, 3, 2)
plt.title('log_modelDate')
sns.distplot(np.log(2021 - data['modelDate']))


In [ ]:
# Логарифмирование не улучшило распредение потому оставим признак без изменений

In [44]:
# Посмотрим на распредение признака productionDate и его логарифмированной версии
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('productionDate')
sns.distplot(data['productionDate'])
plt.subplot(1, 3, 2)
plt.title('log_productionDate')
sns.distplot(np.log(data['productionDate']))


In [ ]:
# Логарифмирование не улучшило распредение потому оставим признак без изменений

In [45]:
# Добавим новый признак - разницу между датой производства и датой создания модели
# Посмотрим на его распределение
sns.distplot(data.productionDate - data.modelDate)

In [ ]:
# Распредение похоже на нормальное потому добавим без изменений

In [46]:
# Добавим признак - пробег в год
sns.distplot(data.mileage/(2021 - data['productionDate']))

In [ ]:
# (data['m_per_y'] > 20000)&(data['sample'] == 1)&(data['mileage']> 200000)

In [50]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output['гараж'] = df_output.description.apply(lambda x: 1 if 'гараж' in x else 0)
    df_output['подарок'] = df_output.description.apply(lambda x: 1 if 'подарок' in x else 0)
    df_output['торг'] = df_output.description.apply(lambda x: 1 if 'торг' in x else 0)
    df_output['шины'] = df_output.description.apply(lambda x: 1 if ('шин' in x) or ('резин' in x) else 0)
    df_output['дилер'] = df_output.description.apply(lambda x: 1 if 'дилер' in x else 0)
    df_output['подогрев'] = df_output.description.apply(lambda x: 1 if 'подогрев' in x else 0)
    df_output['обмен'] = df_output.description.apply(lambda x: 1 if 'обмен' in x else 0)
    df_output['обслуж'] = df_output.description.apply(lambda x: 1 if 'обслуж' in x else 0)
    df_output['срочн'] = df_output.description.apply(lambda x: 1 if 'срочн' in x else 0)
    df_output['полн'] = df_output.description.apply(lambda x: 1 if 'полн' in x else 0)
    df_output['скидк'] = df_output.description.apply(lambda x: 1 if 'скидк' in x else 0)
    df_output['диск'] = df_output.description.apply(lambda x: 1 if 'диск' in x else 0)
    df_output['спорт'] = df_output.description.apply(lambda x: 1 if 'спорт' in x else 0)
    
    df_output.drop(['description','sell_id',], axis = 1, inplace=True)
#     Удалим колонку Владение так как она заполнена менее чем на половину. Это улучшает метрику качества на 0.07%
    df_output.drop(['Владение'], axis = 1, inplace=True)
    categorical_features.remove('Владение')
#     Логарифмируем целевую переменную для придачи ей нормального распределения. 
#     Это улучшает метрику качечтва при использовании CatBoost на 1.4% но не значительно ухудшает на нейросетях потому там будем экспонентой от логарифмированной целевой переменной при обучении
    df_output.price = df_output.price.apply(lambda x: np.log(x) if x !=0 else 0)
    
    
#     Переведем признак engineDisplacement из категориальных в числовые дополнительно прологарифмировав его.
    df_output['engineDisplacement'] = df_output['engineDisplacement'].apply(lambda x: '2.0 LTR' if x == 'undefined LTR' else x)
    df_output['engineDisplacement'] = df_output['engineDisplacement'].apply(lambda x: np.log(float((x.split(' ')[0]))))
    categorical_features.remove('engineDisplacement')
    numerical_features.append('engineDisplacement')
    
    
  #     Переведем признак enginePower из категориальных в числовые дополнительно прологарифмировав его.  
    df_output['enginePower'] = np.log(df_output['enginePower'].str.split().apply(lambda x: int(x[0])))
    categorical_features.remove('enginePower')
    numerical_features.append('enginePower')

#     Сгенерируем 2 новых признака из признака name и удалим столбец name
    df_output['4wd'] = df_output['name'].apply(lambda x: 1 if '4WD' in x else 0)
    categorical_features.append('4wd')
    
    df_output['xdrive'] = df_output['name'].apply(lambda x: 1 if 'xDrive' in x else 0)
    categorical_features.append('xdrive')
    df_output.drop(['name'], axis = 1, inplace=True)
    categorical_features.remove('name')
    
#     Создадим признак - разница между созданием модели и выпуском автомобиля
    df_output['Y_no_sale'] = data.productionDate - data.modelDate
    numerical_features.append('Y_no_sale')
#     Содадим признак - мили в год
    df_output['m_per_y'] = data.mileage/(2021 - data['productionDate'])
    numerical_features.append('m_per_y')
    
    df_output['hard_usage'] = df_output['m_per_y'].apply(lambda x: 1 if x >=20000 else 0)
    categorical_features.append('hard_usage')
    
    df_output['trash'] = data.mileage.apply(lambda x: 1 if x>=300000 else 0)
    categorical_features.append('trash')
    
    df_output['productionDate'] = 2021 - df_output['productionDate']

    
    
#     df_output['retro'] = (df_output['productionDate'] >= 30)
#     df_output['retro'] = df_output['retro'].apply(lambda x: 1 if x is True else 0)
#     categorical_features.append('retro')


#     df_output['cat_productionDate'] = pd.qcut(data['productionDate'], 12, labels=[x for x in range(1,13)])
#     categorical_features.append('cat_productionDate')
    
    df_output['modelDate'] = 2021 - df_output['modelDate']
    # ################### Numerical Features ############################################################## 
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    # Нормализация данных
#     scaler = MinMaxScaler()
#     for column in numerical_features:
#         df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
#     scaler = StandardScaler()
#     for column in numerical_features:
#         df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        
    scaler = RobustScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # тут ваш код не Encoding фитчей
    # ....
    
    
    # ################### Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    df_output['white'] = data.color.apply(lambda x: 1 if 'белый' in x else 0)
    categorical_features.append('white')
    
    # ################### Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    df_output.drop(['vehicleConfiguration'], axis = 1, inplace=True)
    return df_output

In [51]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

## Split data

In [52]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [53]:
test_data.info()

# Model 2: CatBoostRegressor

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [55]:
model = CatBoostRegressor(iterations = 5000,
#                           depth=5,
#                           learning_rate = 0.022,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
#                           max_leaves=32
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
#          plot=True
         )

In [56]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_catboost))*100:0.2f}%")



### Submission

In [57]:
sub_predict_catboost = np.exp(model.predict(X_sub))
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

# Model 3: Tabular NN

Построим обычную сеть:

In [58]:
X_train.head(5)

## Simple Dense NN

In [59]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.3))
model.add(L.Dense(128, activation="relu"))
model.add(L.Dense(1, activation="linear"))

In [ ]:
# model.summary()

In [60]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [61]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### Fit

In [62]:
history = model.fit(X_train, np.exp(y_train),
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, np.exp(y_test)),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [63]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [64]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [65]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn1[:,0]))*100:0.2f}%")

In [66]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

In [67]:
new_pred = (test_predict_catboost + test_predict_nn1[:,0])/2
print(f"TEST mape: {(mape(np.exp(y_test), new_pred))*100:0.2f}%")

In [68]:
sample_submission['price'] = (sub_predict_nn1[:,0] + sub_predict_catboost)/2
sample_submission.to_csv('nn1_cat_submission.csv', index=False)

## Проброс признака

In [72]:
# FEATURE Input
# Iput
sport = L.Input(shape=[1], name="sport")

emb_sport = L.Embedding(
    len(X.спорт.unique().tolist())+1, 20
)(sport)

f_sport = L.Flatten()(emb_sport)

In [74]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.3))
model.add(L.Dense(128, activation="relu"))
#model.add(L.Dense(1, activation="linear"))

In [75]:
combinedInput = L.concatenate([model.output, f_sport,])
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)
model = Model(inputs=[model.input, sport], outputs=head)

In [76]:
model.summary()

In [78]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
#%%time
history = model.fit(
     [X_train, X_train.спорт.values],
     np.exp(y_train),
     batch_size=512,
     epochs=500,
     validation_data=([X_test, X_test.спорт.values], np.exp(y_test)),
     callbacks=callbacks_list)

In [79]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [83]:
# model.load_weights('../working/best_model.hdf5')
test_predict_nn_sport = model.predict([X_test, X_test.спорт.values])
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn_sport[:,0]))*100:0.2f}%")

In [84]:
new_pred_bonus = (test_predict_catboost + test_predict_nn_sport[:,0])/2
print(f"TEST mape: {(mape(np.exp(y_test), new_pred_bonus))*100:0.2f}%")

In [89]:
sub_predict_nn_sport = model.predict([X_sub, X_sub.спорт.values])

sample_submission['price'] = (sub_predict_nn_sport[:,0] + sub_predict_catboost) / 2
sample_submission.to_csv('nn_b_submission.csv', index=False)

In [ ]:
# new_pred

Рекомендации для улучшения Model 3:    
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'), поэтому эти данные можно удалить. Затем следует еще сильнее сократить размерность, например, выделив наличие xDrive в качестве отдельного признака.

# Model 4: NLP + Multiple Inputs

In [90]:
data.description

In [91]:
!pip install pymorphy2

In [92]:
import pymorphy2  # $ pip install pymorphy2

def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
functors_pos = {'INTJ', 'CONJ', 'PREP'}  # function words

In [93]:
data.description = data.description.apply(lambda x: " ".join([word for word in x.split() if pos(word) not in functors_pos]))

In [94]:
morph = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+●•✅☑️☛"

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        token = token.strip()
        token = morph.normal_forms(token)[0]
        tokens.append(token)
    return ' '.join(tokens)

In [95]:
data['description'] = df_NLP.apply(
    lambda df_NLP: lemmatize(df_NLP.description), axis=1)

In [96]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [97]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

### Tokenizer

In [98]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n•–∙—″☑️✔➥●·✅☛———————————————————————————')
tokenize.fit_on_texts(data.description)

In [99]:
tokenize.word_index

In [100]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [101]:
# # вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

### RNN NLP

In [102]:
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Activation
from tensorflow.keras import activations

In [103]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
#model_nlp.add(Bidirectional(LSTM(256, return_sequences=True)))
model_nlp.add(L.Dropout(0.2))
#model_nlp.add(Bidirectional(LSTM(128)))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.2))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.15))

### MLP

In [104]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.35))

### Multiple Inputs NN

In [105]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [106]:
model.summary()

### Fit

In [107]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [108]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [109]:
history = model.fit([text_train_sequences, X_train], np.exp(y_train),
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], np.exp(y_test)),
                    callbacks=callbacks_list
                   )

In [110]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [111]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [112]:
model.predict([text_test_sequences, X_test])

In [113]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn2[:,0]))*100:0.2f}%")

In [114]:
new_pred_1 = (test_predict_catboost + test_predict_nn2[:,0] + test_predict_nn1[:,0])/3
print(f"TEST mape: {(mape(np.exp(y_test), new_pred_1))*100:0.2f}%")

In [116]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = (sub_predict_nn2[:,0] + sub_predict_catboost + sub_predict_nn_sport[:,0])/3
sample_submission.to_csv('nn2_1_cat_submission.csv', index=False)

Идеи для улучшения NLP части:
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например, сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа
Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста

# Model 5: Добавляем картинки

### Data

In [117]:
 # убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [118]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [119]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)


#пример взят из официальной документации: https://albumentations.readthedocs.io/en/latest/examples.html
augmentation = Compose([
    HorizontalFlip(),
    OneOf([
        IAAAdditiveGaussianNoise(),
        GaussNoise(),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),
        IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(),
    ], p=0.3),
    HueSaturationValue(p=0.3),
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [120]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
            augment_dict = augmentation(image = images[i])
            augmented_image = augment_dict['image']
            augmented_images[i] = augmented_image
    print('')
    return augmented_images

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [121]:
# # NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [139]:
X

In [146]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, sport, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text, sport), price

def tf_process_val_dataset_element(image, table_data, text, sport, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text, sport), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], X_train.спорт.values, np.exp(y_train)
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], X_test.спорт.values, np.exp(y_test)
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], X_sub.спорт.values, np.exp(y_sub)
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();


### Строим сверточную сеть для анализа изображений без "головы"

In [123]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [147]:
#строим нейросеть для анализа табличных данных


tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.3),
    L.Dense(128, activation = 'relu')
    ])

In [148]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])



In [149]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output, f_sport])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input, sport], outputs=head)
model.summary()

In [150]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [151]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [152]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [153]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [154]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [155]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn3[:,0]))*100:0.2f}%")

In [156]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)


#### Общие рекомендации:
* Попробовать разные архитектуры
* Провести более детальный анализ результатов
* Попробовать различные подходы в управление LR и оптимизаторы
* Поработать с таргетом
* Использовать Fine-tuning

#### Tabular
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно еще сильнее сократить размерность, например выделив наличие xDrive в качестве отдельного признака.

* Поработать над Feature engineering



#### NLP
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа
Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста



#### CV
* Попробовать различные аугментации
* Fine-tuning

# Blend

In [ ]:
# blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
# print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
# blend_sub_predict = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
# sample_submission['price'] = blend_sub_predict
# sample_submission.to_csv('blend_submission.csv', index=False)

# Model Bonus: проброс признака

In [ ]:
# # MLP
# model_mlp = Sequential()
# model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
# model_mlp.add(L.Dropout(0.5))
# model_mlp.add(L.Dense(256, activation="relu"))
# model_mlp.add(L.Dropout(0.5))

In [ ]:
# # FEATURE Input
# # Iput
# engineDisplacement = L.Input(shape=[1], name="engineDisplacement")

# emb_engineDisplacement = L.Embedding(
#     len(X.engineDisplacement.unique().tolist())+1, 20
# )(engineDisplacement)

# f_engineDisplacement = L.Flatten()(emb_engineDisplacement)

In [ ]:
# combinedInput = L.concatenate([model_mlp.output, f_engineDisplacement,])
# head = L.Dense(64, activation="relu")(combinedInput)
# head = L.Dense(1, activation="linear")(head)
# model = Model(inputs=[model_mlp.input, engineDisplacement], outputs=head)

In [ ]:
# model.summary()

In [ ]:
# optimizer = tf.keras.optimizers.Adam(0.01)
# model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
# optimizer = tf.keras.optimizers.Adam(0.01)
# model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

# %%time
# history = model.fit(
#     [X_train, X_train.engineDisplacement.values],
#     np.exp(y_train),
#     batch_size=512,
#     epochs=500,
#     validation_data=([X_test, X_test.engineDisplacement.values], np.exp(y_test)),
#     callbacks=callbacks_list
)

In [ ]:
# model.load_weights('../working/best_model.hdf5')
# test_predict_nn_bonus = model.predict([X_test, X_test.engineDisplacement.values])
# print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn_bonus[:,0]))*100:0.2f}%")

In [ ]:
# new_pred_bonus = (test_predict_catboost + test_predict_nn_bonus[:,0])/2
# print(f"TEST mape: {(mape(np.exp(y_test), new_pred_bonus))*100:0.2f}%")

In [ ]:
# sub_predict_nn_bonus = model.predict(X_sub)
# sample_submission['price'] = sub_predict_nn_bobus[:,0]
# sample_submission.to_csv('nn_b_submission.csv', index=False)

In [ ]:
# sample_submission['price'] = (sub_predict_nn_bonus[:,0] + sub_predict_catboost)/2
# sample_submission.to_csv('nn_b_cat_submission.csv', index=False)